# Model fine-tuning for Bio-image Analysis tasks
In this notebook we use `bia_bob` infrastructure to fine-tune a chatGPT 3.5 model specifically for Bio-image Analysis tasks.

A little warning: Fine-tuning costs money. If you [use it the wrong way](https://x.com/haesleinhuepf/status/1718575819298103336?s=20), running this notebook can become expensive.              

In [1]:
from bia_bob import FineTuningFromNotebooks
from bia_bob._utilities import filter_out_blacklist
import time
import os

## Training data

We use Python Jupyter Notebooks as training data. These notebooks must have high quality explanatory markdown cells between code cells. For demonstration purposes, we use a subset of the [BioImage Analysis notebooks](https://haesleinhuepf.github.io/BioImageAnalysisNotebooks/intro.html).

First, we make a list of notebook files.

In [2]:
notebooks = []
directory = "C:/structure/code/BioImageAnalysisNotebooks/docs/"

for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".ipynb") and ".ipynb_checkpoints" not in root:
            # print(os.path.join(root, file))

            notebooks.append(os.path.join(root, file))

f"{len(notebooks)} notebooks listed"

'240 notebooks listed'

We then filter out some notebooks using a blacklist of folder or filenames.

In [3]:
notebooks = filter_out_blacklist(notebooks, [
    "python_basics",
    "prompt_engineering",
    "sustainable_code",
    "sql",  
])

f"{len(notebooks)} notebooks remaining"

'209 notebooks remaining'

We now initialize finetuning. Under the hood, the notebooks are parsed and conversations are extracted.

In [4]:
fine_tuning = FineTuningFromNotebooks(notebooks)

f"{len(fine_tuning._training_data)} conversations extracted"

'1253 conversations extracted'

We could look into a single conversation like this:

In [5]:
# fine_tuning._training_data[50]

We now filter out conversations which contain words from a black list.

In [6]:
fine_tuning._training_data = filter_out_blacklist(fine_tuning._training_data, [
    "napari",
    "nbscreenshot",
    "def ",
    "print",
    "openai",
    "https://"
])

f"{len(fine_tuning._training_data)} conversations remaining"

'749 conversations remaining'

Another way for limiting training data (to spare money) is to sample randomly.

In [7]:
import random
fine_tuning._training_data = random.sample(fine_tuning._training_data, 100)

f"{len(fine_tuning._training_data)} conversations remaining"

'100 conversations remaining'

## Fine tuning
We start the fine-tuning ...

In [8]:
fine_tuning.train()

... and wait for it to be finished.

In [9]:
while not fine_tuning.is_trained():
    print("Still training")
    time.sleep(100)

Still training
Still training
Still training
Still training
Still training
Still training
Still training
Still training


Afterwards, we can print out the name of the fine-tuned model.

In [10]:
model_name = fine_tuning.trained_model_name()
model_name

'ft:gpt-3.5-turbo-0613:personal::8EyMmUpY'

We can then use this model in `bia_bob`. Note: If you copy-paste the name of this model, you can reuse it any time. However, you cannot share it with others without sharing also the API-Key.

If there is an error `ServiceUnavailableError: The server is overloaded or not ready yet` in the following, the model is not available yet. Try again some minutes later.

In [ ]:
from bia_bob import bob
bob.initialize(model_name)

In [ ]:
%%bob 
load the image c:/structure/data/blobs.tif,
label the objects in it, 
expand the objects using Voronoi-Tesselation, and
draw a mesh between the centroids of the segmented objects.

Make a plan first, before you start and don't forget to add the necessary import statements on top.